## Day 20

https://adventofcode.com/2023/day/20

In [21]:
class Module():
    def __init__(self,name,typ,outputs):
        self.name = name
        self.typ = typ
        self.outputs = outputs
        # initialize memory
        if self.typ == "%": # Flip-flop 
            self.memory = 0
        else: # Conjunction
            self.memory = {} # dictionary of all connections and their value. Needs to be initialises.
            
    def __repr__(self):
        return "{} {:4s} | OUT: {:20s} | MEM: {}".format(self.typ,self.name,str(self.outputs),self.memory)

    def getpulse(self,origin,pulse):
        if self.typ == "&": # conjunction
            self.memory[origin] = pulse # update memory with received pulse
            if all([ v==1 for i,v in self.memory.items() ]):
                return [ (o,0) for o in self.outputs ]
            else:
                return [ (o,1) for o in self.outputs ]
        else: # Flip-flop
            # If a flip-flop module receives a high pulse, it is ignored and nothing happens. 
            if pulse==0:
                self.memory = abs(self.memory-1) # flips status
                return [ (o,self.memory) for o in self.outputs ]
            else:
                return []
    
def readInput20(infile):
    modules = {} 
    broadcasts = []
    with open(infile) as f:
        for l in f.read().strip().splitlines():
            name,outputs = l.split(" -> ")
            if name == "broadcaster":
                broadcasts = outputs.split(", ")
            else:
                typ = name[0]
                name = name[1:]
                modules[name] = Module(name,typ,outputs.split(", "))
        # initialize conjuctions inputs
        for n,m in modules.items():
            for o in m.outputs:
                if o!="output" and o in modules.keys() and modules[o].typ == "&": # is a conjuction
                    modules[o].memory[n] = 0
        return modules, broadcasts
        
modules, broadcasts = readInput20("examples/example20-2.txt")

#print(broadcasts)
for n,m in modules.items():
    print(m)

% a    | OUT: ['inv', 'con']       | MEM: 0
& inv  | OUT: ['b']                | MEM: {'a': 0}
% b    | OUT: ['con']              | MEM: 0
& con  | OUT: ['output']           | MEM: {'a': 0, 'b': 0}


In [22]:
from queue import Queue

def push_button(infile,npush=1000,verbose=False,part=1):

    modules, broadcasts = readInput20(infile)
    
    low = 0
    high = 0
    rx = -1
    
    i = 0
    
    #for i in range(npush): # every cycle is a button push
    while True:
        
        i+=1
        rx = -1
        
        # push button
        if verbose: print("button 0 -> B")
        low+=1
        
        # enqueue broadcast signals
        q = Queue()
        for target in broadcasts:
            q.put(("B",target,0)) # origin, target, pulse
            
        # process all pulses in queue until empty
        while not q.empty():
            
            origin,target,pulse = q.get()      
            if verbose: print(origin,pulse,"->",target)

            if pulse==0:
                low+=1
            else:
                high+=1 
            
            if target in modules.keys(): # ignore disconnected modules (e.g. rx) and the 'output' node
                reply = modules[target].getpulse(origin,pulse)
                for t,p in reply:
                    q.put((target,t,p))

            if target=="rx":
                rx = pulse

        if part==1 and i==npush:
            break
        
        if part==2: # Part 2 brute forcing attempt (not worth!)
            if i%10_000==0:
                print("*",i)
            if rx==0:
                print(i+1)
                return i+1
        
    if verbose: print("LOW:",low,"HIGH:",high)
    return low*high

def part1(infile):
    return push_button(infile)

In [23]:
push_button("examples/example20-1.txt",1,True)

button 0 -> B
B 0 -> a
B 0 -> b
B 0 -> c
a 1 -> b
b 1 -> c
c 1 -> inv
inv 0 -> a
a 0 -> b
b 0 -> c
c 0 -> inv
inv 1 -> a
LOW: 8 HIGH: 4


32

In [24]:
print("Test 1-1:",part1("examples/example20-1.txt")) # 32000000
print("Test 1-2:",part1("examples/example20-2.txt")) # 670984704
print("Part 1  :",part1("AOC2023inputs/input20.txt")) # 670984704

Test 1-1: 32000000
Test 1-2: 11687500
Part 1  : 670984704


### Part 2

The unconnect module I spotted in Part 1 (`nx`) is now the center of Part 2. Simply bruteforcing part 1 solution does not seem to work, some reverse engineering of the module network seems to be needed.

Observations:

* `rx` is connected only to `zr`
* `zr` is a conjunction connected to `gc`, `sz`, `cm`, `xf`.

In [53]:
modules, broadcasts = readInput20("AOC2023inputs/input20.txt")

print(modules['zr'])

print(modules['gc'])
print(modules['sz'])
print(modules['cm'])
print(modules['xf'])

& zr   | OUT: ['rx']               | MEM: {'gc': 0, 'sz': 0, 'cm': 0, 'xf': 0}
& gc   | OUT: ['zr']               | MEM: {'dn': 0}
& sz   | OUT: ['zr']               | MEM: {'ms': 0}
& cm   | OUT: ['zr']               | MEM: {'ks': 0}
& xf   | OUT: ['zr']               | MEM: {'tc': 0}


In order to get a low pulse to `rx`, I need `zr` to issue a low-pulse. Since `zx` is a conjuction, in order to issue a low pulse, all 4 inputs need to be high. 

I suspect that these inputs will cycle between being low and high with very different cycles (probably with coprime periods): I could search for these cycles, then compute their least commom multiple...

In [54]:
def search_cycles(infile,feed_nx='zr'):
    modules, broadcasts = readInput20(infile)
    inputs_fire = dict(modules[feed_nx].memory)    
    i = 0
    while True:
        # --- push button start
        i+=1        
        q = Queue()
        for target in broadcasts:
            q.put(("B",target,0))
        while not q.empty():   
            origin,target,pulse = q.get()
            ### ----
            if target == feed_nx and pulse==1:
                inputs_fire[origin] = i # store first button push where the zr inputs fire (assuming it's a period)
                if all(inputs_fire.values()):
                    return inputs_fire
            ### ----
            if target in modules.keys():
                reply = modules[target].getpulse(origin,pulse)
                for t,p in reply:
                    q.put((target,t,p))
        # --- push button end  
    return

In [55]:
inputs_fire = search_cycles("AOC2023inputs/input20.txt",feed_nx='zr')
inputs_fire

{'gc': 3853, 'sz': 4093, 'cm': 4091, 'xf': 4073}

In [57]:
import numpy as np

def part2():
    inputs_fire = search_cycles("AOC2023inputs/input20.txt",feed_nx='zr')
    periods = list(inputs_fire.values())
    period_nx = np.lcm.reduce(periods)
    return period_nx

print("Part 2:",part2())

Part 2: 262775362119547
